In [1]:

from glob import glob
import gzip
import contextlib
import json
import re
import zipfile
import tarfile
import tempfile
import os
import pathlib
import subprocess
import notebook_helper

# depending on which test runner was used, the collection archive may be either a zip (local docker & exec runner),
# or a tar.gz file (k8s). Unfortunately, the zipfile and tarfile modules are different species of waterfowl,
# so we duck typing doesn't help. So this method extracts whichever one we have to a temp directory and
# returns the path to the temp dir.
# use as a context manager, so the temp dir gets deleted when we're done:
#   with open_archive(archive_path) as a:
#     files = glob(a + '/**/tracer-output')
@contextlib.contextmanager
def open_archive(archive_path):
    # zipfile and tarfile both have an extractall method, at least
    #print(archive_path)
    if zipfile.is_zipfile(archive_path):
        z = zipfile.ZipFile(archive_path)
    else:
        z = tarfile.open(archive_path, 'r:gz')

    #print(z)
    with tempfile.TemporaryDirectory(prefix='pubsub-tg-archive-') as d:
        print(d)
        z.extractall(path=d)
        yield d
   
def concat_files(names, outfile):
    for name in names:
        with open(name, 'rb') as f:
            outfile.write(f.read())

def mkdirp(dirpath):
    pathlib.Path(dirpath).mkdir(parents=True, exist_ok=True)

PEER_INFO_PATTERN = re.compile(
    r'Host peer ID: ([0-9a-zA-Z]+), seq (\d+)')


def extract_peer_info(run_out):
    infos = []
    with open(run_out, 'rt') as f:
        for line in f.readlines():
            m = PEER_INFO_PATTERN.search(line)
            if m:
                pid = m.group(1)
                seq = int(m.group(2))
                node_type = m.group(3)
                node_type_seq = int(m.group(4))
                node_index = int(m.group(5))
                node_index_bound = int(m.group(6))
                infos.append(
                    {'peer_id': pid,
                     'seq': seq})
    if len(infos) == 0:
        print('warning: no peer info found in {}'.format(run_out))
    return infos

def extract_timing_info(run_out, node_type):
    times = dict(t_warm=0, t_connect=0, t_run=0, t_cool=0, t_complete=0)

    with open(run_out, 'rt') as f:
        for line in f.readlines():
            try:
                obj = json.loads(line)
            except BaseException as err:
                print("error parsing run output: ", err)
                continue
            if 'ts' not in obj or 'event' not in obj or obj['event'].get('type', '') != 'message':
                continue
            msg = obj['event']['message']
            ts = obj['ts']
            if re.match(r'connecting to peers.*', msg):
                times['t_connect'] = ts
                continue

            # the rest of the times are only logged by honest peers
            if node_type != 'honest':
                continue
            if re.match(r'Wait for .* warmup time', msg):
                times['t_warm'] = ts
                continue
            if re.match(r'Wait for .* run time', msg):
                times['t_run'] = ts
                continue
            if re.match(r'Run time complete, cooling down.*', msg):
                times['t_cool'] = ts
                continue
            if msg == 'Cool down complete':
                times['t_complete'] = ts
                continue
            
def extract_peer_and_timing_info(run_out_files):
    entries = []
    for filename in run_out_files:
        infos = extract_peer_info(filename)
        print(infos)
        if infos is None or len(infos) == 0:
            continue

        # FIXME: we only get the timing info for the first peer in the container
        # unfortunately, getting the times for individual peers requires code changes
        times = extract_timing_info(filename, infos[0].get('type', 'unknown'))
        for info in infos:
            info.update(times)
            entries.append(info)
    return entries

# sugar around recursive glob search
def find_files(dirname, filename_glob):
    path = '{}/**/{}'.format(dirname, filename_glob)
    return glob(path, recursive=True)

def aggregate_output(output_zip_path, out_dir):
    topology = dict()

    print(output_zip_path)
    with open_archive(output_zip_path) as archive:
        print(archive)
        tracefiles = find_files(archive, 'tracer-output*')
        names = [f for f in tracefiles if 'full' in f]
        if len(names) > 0:
            with gzip.open(os.path.join(out_dir, 'full-trace.bin.gz'), 'wb') as gz:
                concat_files(names, gz)

        names = [f for f in tracefiles if 'filtered' in f]
        if len(names) > 0:
            with gzip.open(os.path.join(out_dir, 'filtered-trace.bin.gz'), 'wb') as gz:
                concat_files(names, gz)

        print(archive)
        # get peer id -> seq mapping & timing info from run.out files
        #names = find_files(archive, 'run.out')
        #info = extract_peer_and_timing_info(names)
        #dest = os.path.join(out_dir, 'peer-info.json')
        #with open(dest, 'wt') as f:
        #    json.dump(info, f)

def extract_test_outputs(test_output_zip_path, output_dir=None):
    if output_dir is None or output_dir == '':
        output_dir = os.path.join(os.path.dirname(test_output_zip_path), 'analysis')

    raw_output_dir = os.path.join(output_dir, 'raw-data')
    mkdirp(raw_output_dir)
    aggregate_output(test_output_zip_path, raw_output_dir)
    run_tracestat(raw_output_dir)

    #if convert_to_pandas:
    print('converting data to pandas format...')
    pandas_dir = os.path.join(output_dir, 'pandas')

    notebook_helper.to_panda(raw_output_dir, pandas_dir)
    #if prep_notebook:
    #    prepare_analysis_notebook(analysis_dir=output_dir)
    return output_dir

def run_tracestat(tracer_output_dir):
    full = os.path.join(tracer_output_dir, 'full-trace.bin.gz')
    filtered = os.path.join(tracer_output_dir, 'filtered-trace.bin.gz')
    if os.path.exists(full):
        tracer_output = full
    elif os.path.exists(filtered):
        tracer_output = filtered
    else:
        print('no event tracer output found, skipping tracestat')
        return

    print('running tracestat on {}'.format(tracer_output))
    try:
        cmd = ['tracestat', '-cdf', tracer_output]
        p = subprocess.run(cmd, capture_output=True, text=True, check=True)
    except BaseException as err:
        print('error calling tracestat: ', err)
        return

    # split output into summary and latency CDF
    [summary, cdf] = p.stdout.split('=== Propagation Delay CDF (ms) ===')

    with open(os.path.join(tracer_output_dir, 'tracestat-summary.txt'), 'w', encoding='utf8') as f:
        f.write(summary)
    with open(os.path.join(tracer_output_dir, 'tracestat-cdf.txt'), 'w', encoding='utf8') as f:
        f.write(cdf)

    #print(cdf)


extract_test_outputs('/home/sergi/workspace/gossip_test/clecdbalo7jb9ut0vri0.tgz','/home/sergi/workspace/gossip_test/clecdbalo7jb9ut0vri0')

/home/sergi/workspace/gossip_test/clecdbalo7jb9ut0vri0.tgz
/home/sergi/seal/tmpo2/pubsub-tg-archive-6_z0612r
/home/sergi/seal/tmpo2/pubsub-tg-archive-6_z0612r
/home/sergi/seal/tmpo2/pubsub-tg-archive-6_z0612r
running tracestat on /home/sergi/workspace/gossip_test/clecdbalo7jb9ut0vri0/raw-data/full-trace.bin.gz
converting data to pandas format...
converting latency cdf to pandas...
writing cdf pandas data to /home/sergi/workspace/gossip_test/clecdbalo7jb9ut0vri0/pandas/cdf.csv


'/home/sergi/workspace/gossip_test/clecdbalo7jb9ut0vri0'